<a href="https://colab.research.google.com/github/baldeoJV/CCADMACL_EXERCISES_COM222ML/blob/main/Exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 2: Use Gradient Boost for Regression

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
Submit your results to:
https://www.kaggle.com/competitions/playground-series-s4e12/overview



In [151]:
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

## Dataset
Train, test and sample submission file can be found in this link
https://www.kaggle.com/competitions/playground-series-s4e12/data

## 1. Load the Data

In [186]:
#data
test = pd.read_csv('test.csv')
sf = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')

## 2. Perform Data preprocessing

######train.csv

In [187]:
train['Age'].fillna(train['Age'].median(), inplace=True)
train['Annual Income'].fillna(train['Annual Income'].median(), inplace=True)
train['Health Score'].fillna(train['Health Score'].median(), inplace=True)
train['Credit Score'].fillna(train['Credit Score'].median(), inplace=True)
train['Vehicle Age'].fillna(train['Vehicle Age'].median(), inplace=True)
train['Insurance Duration'].fillna(train['Insurance Duration'].median(), inplace=True)
train['Number of Dependents'].fillna(train['Number of Dependents'].mode()[0], inplace=True)
train['Previous Claims'].fillna(train['Previous Claims'].mode()[0], inplace=True)
train['Marital Status'].fillna(train['Marital Status'].mode()[0], inplace=True)
train['Occupation'].fillna(train['Occupation'].mode()[0], inplace=True)
train['Customer Feedback'].fillna(train['Customer Feedback'].mode()[0], inplace=True)

<ipython-input-187-2e93351504fd>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Age'].fillna(train['Age'].median(), inplace=True)
<ipython-input-187-2e93351504fd>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

In [188]:
# Define categorical and numerical features
categorical_features = train.select_dtypes(
   include=["object"]
).columns.tolist()

numerical_features = train.select_dtypes(
   include=["float64", "int64"]
).columns.tolist()


In [189]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocessor = ColumnTransformer(
   transformers=[
       ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features),
       ("num", StandardScaler(), numerical_features),
   ]
)

######test.csv

In [190]:
test['Age'].fillna(test['Age'].median(), inplace=True)
test['Annual Income'].fillna(test['Annual Income'].median(), inplace=True)
test['Health Score'].fillna(test['Health Score'].median(), inplace=True)
test['Credit Score'].fillna(test['Credit Score'].median(), inplace=True)
test['Vehicle Age'].fillna(test['Vehicle Age'].median(), inplace=True)
test['Insurance Duration'].fillna(test['Insurance Duration'].median(), inplace=True)
test['Number of Dependents'].fillna(test['Number of Dependents'].mode()[0], inplace=True)
test['Previous Claims'].fillna(test['Previous Claims'].mode()[0], inplace=True)
test['Marital Status'].fillna(test['Marital Status'].mode()[0], inplace=True)
test['Occupation'].fillna(test['Occupation'].mode()[0], inplace=True)
test['Customer Feedback'].fillna(test['Customer Feedback'].mode()[0], inplace=True)

<ipython-input-190-5f69a99dfa45>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Age'].fillna(test['Age'].median(), inplace=True)
<ipython-input-190-5f69a99dfa45>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [191]:
categorical_features = test.select_dtypes(
   include=["object"]
).columns.tolist()

numerical_features = test.select_dtypes(
   include=["float64", "int64"]
).columns.tolist()

In [192]:
preprocessor = ColumnTransformer(
   transformers=[
       ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features),
       ("num", StandardScaler(), numerical_features),
   ]
)

######corellation

#####Splits

In [193]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1200000 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1200000 non-null  float64
 4   Marital Status        1200000 non-null  object 
 5   Number of Dependents  1200000 non-null  float64
 6   Education Level       1200000 non-null  object 
 7   Occupation            1200000 non-null  object 
 8   Health Score          1200000 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       1200000 non-null  float64
 12  Vehicle Age           1200000 non-null  float64
 13  Credit Score          1200000 non-null  float64
 14  Insurance Duration    1200000 non-

In [195]:
# Split data into train and test sets

x = train.drop('Premium Amount', axis=1)

y = train['Premium Amount']

X_train, X_test, y_train, y_test = train_test_split(
   train, y, test_size=0.2, random_state=42
)

## 3. Create a Pipeline with Params

In [196]:
params = {
    "n_estimators": 500,
    "max_depth": 10,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": 'friedman_mse',
}

In [197]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
   [
       ("preprocessor", preprocessor),
       ("classifier", GradientBoostingRegressor(random_state=42)),
   ]
)

## 4. Train the Model

In [198]:
# put your answer here
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Gender', 'Marital Status',
                                                   'Education Level',
                                                   'Occupation', 'Location',
                                                   'Policy Type',
                                                   'Policy Start Date',
                                                   'Customer Feedback',
                                                   'Smoking Status',
                                                   'Exercise Frequency',
                                                   'Property Type']),
                                                 ('num', StandardScaler(),
                                                  ['id', 'Age', 'Annual Income',
                                                   'Number of Dependents',
                                                   'Health Score',
                                                   'Previous Claims',
                                                   'Vehicle Age',
                                                   'Credit Score',
                                                   'Insurance Duration'])])),
                ('classifier', GradientBoostingRegressor(random_state=42))])

## 5. Evaluate the Model

In [199]:
# put your answer here
from sklearn.metrics import root_mean_squared_log_error

# Predict on the test set
y_pred = pipeline.predict(X_test)


In [200]:
rmse = root_mean_squared_log_error(y_test, y_pred)
print(f"RMSE: {rmse}")

RMSE: 1.1614373927561275


## Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [201]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    800000 non-null  int64  
 1   Age                   800000 non-null  float64
 2   Gender                800000 non-null  object 
 3   Annual Income         800000 non-null  float64
 4   Marital Status        800000 non-null  object 
 5   Number of Dependents  800000 non-null  float64
 6   Education Level       800000 non-null  object 
 7   Occupation            800000 non-null  object 
 8   Health Score          800000 non-null  float64
 9   Location              800000 non-null  object 
 10  Policy Type           800000 non-null  object 
 11  Previous Claims       800000 non-null  float64
 12  Vehicle Age           800000 non-null  float64
 13  Credit Score          800000 non-null  float64
 14  Insurance Duration    800000 non-null  float64
 15  

In [202]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              800000 non-null  int64  
 1   Premium Amount  800000 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 12.2 MB


In [203]:
id = sf['id']
y_pred = pipeline.predict(test)

# Create a submission DataFrame
sf = pd.DataFrame({
    'id': id,
    'Premium Amount': y_pred
})

# Save the submission DataFrame to a CSV file
sf.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
